In [554]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import os
from pyproj import Proj
import TEAK_class_declarations
import math
import importlib
import pickle
from scipy.ndimage import gaussian_filter
from scipy.ndimage import convolve

objects_path = "/data/shared/src/STV/NEON_TEAK/allen/object_data"
output_path = '/data/shared/src/STV/NEON_TEAK/allen/figs/model_output/'
plot_LAI_path = '/data/shared/src/STV/NEON_TEAK/allen/data/NEON_struct-plant/matching_shapefiles_tiffs.csv'
# plot_LAI_path = "/data/shared/src/STV/NEON_TEAK/allen/figs/model_output/TEAK_sites_LAI_Average_only.csv"
orig_gort_path = '/data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_13'
# orig_gort_path = '/data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_11_small_footprint'

plot_LAI_df = pd.read_csv(plot_LAI_path)

section_range = 0.1  # specifies section size.  (section is per 0.1 meters)

def load_objects_from_file(filename, path=""):
    # Adjust the filename to include the subfolder
    if path:
        filename = os.path.join(path, filename)
        
    # Load the objects from file
    with open(filename, 'rb') as f:
        objects_list = pickle.load(f)
    
    return objects_list

def get_date_subfolder():
    current_date = datetime.now()
    return current_date.strftime('%Y_%m_%d')

date_subfolder = get_date_subfolder()

In [555]:
teak_tree_objects_100 = load_objects_from_file("tree_objects_100.pkl", objects_path)
teak_tree_objects_400 = load_objects_from_file("tree_objects_400.pkl", objects_path)
teak_plot_objects_100 = load_objects_from_file("plot_objects_100.pkl", objects_path)
teak_plot_objects_400 = load_objects_from_file("plot_objects_400.pkl", objects_path)

pgap_list = load_objects_from_file("TEAK_pgap_list.pkl", objects_path)

In [556]:
# print(len(pgap_list))

def get_pgap_by_plot_id(pgap_list, target_plot_id):
    # Search for the target plot_id
    for pgap_obj, plot_id in pgap_list:
        if plot_id == target_plot_id:
            return pgap_obj  # Return the matching object

    # If not found
    print(f"Plot ID {target_plot_id} not found.")
    return None
        
pgap = get_pgap_by_plot_id(pgap_list, "TEAK_044")

mask = (pgap.height > 0.1) & (~np.isnan(pgap.foliageDensity)) # exclude all data below height 0.1, as they are too large.  Also filter out NaN values
pgap_ht = pgap.height[mask]
pgap_fp = pgap.foliageDensity[mask]
pgap_gap = pgap.gap[mask]

# Calculate the height differences (dz)
height_diffs = np.abs(np.diff(pgap_ht, prepend=pgap_ht[0]))

pgap_acc_LAI = np.cumsum(pgap_fp * height_diffs)# Cumulative sum in reverse order

print(pgap_ht)
print(height_diffs)
print(pgap_fp)
print(pgap_acc_LAI)

[37. 36. 35. 34. 33. 32. 31. 30. 29. 28. 27. 26. 25. 24. 23. 22. 21. 20.
 19. 18. 17. 16. 15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.
  1.]
[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0.         0.00400401 0.00401204 0.00603319 0.01009593 0.0081136
 0.01223245 0.0164272  0.01864333 0.02092069 0.02752823 0.03224143
 0.03938858 0.0379264  0.03407237 0.04166817 0.04494571 0.07557267
 0.04777097 0.05677801 0.05036714 0.04891548 0.05579162 0.06907161
 0.04740963 0.05471466 0.05942579 0.06452172 0.07006555 0.06557965
 0.08970113 0.09007151 0.11048174 0.17808367 0.20530831 0.08533289
 0.12156978]
[0.         0.00400401 0.00801604 0.01404923 0.02414516 0.03225876
 0.04449122 0.06091841 0.07956174 0.10048243 0.12801066 0.16025209
 0.19964067 0.23756707 0.27163945 0.31330762 0.35825333 0.433826
 0.48159697 0.53837498 0.58874212 0.6376576  0.69344923 0.76252084
 0.80993047 0.86464512 0.92407092 0.98859264 1.0586581

In [ ]:

def calculate_Li_dz(plot, trees, section_range, plot_area):
    num_trees = len(plot.tree_individualID)
    max_tree_height = max(tree.height for tree in trees if tree.individualID in plot.tree_individualID)

    plot.tree_height_array = np.arange(0, max_tree_height + section_range, section_range)
    plot.Li_dz_indiv = np.zeros((num_trees, len(plot.tree_height_array)))

    LAI = plot.LAI

    plot_crown_vols = []
    
    for index, tree_id in enumerate(plot.tree_individualID):
        tree = next((t for t in trees if t.individualID == tree_id), None)
        crown_volume = tree.crown_volume
        plot_crown_vols.append(crown_volume)
    
    total_crown_volume = sum(plot_crown_vols)
    
    Fa = (LAI/total_crown_volume) * plot_area
    print(f"plot id: {plot.plotid}, num_trees: {num_trees}, max_tree_height: {max_tree_height}, LAI: {LAI}, total crown vol: {total_crown_volume}, Fa: {Fa}")

    for index, tree_id in enumerate(plot.tree_individualID):
        # Find the tree with the matching individualID
        tree = next((t for t in trees if t.individualID == tree_id), None)

        tree_height = tree.height
        crown_base_height = tree.baseCrownHeight
        crown_radius = tree.horiz_crown_radius
        crown_center_height = tree.crownCenterHeight
        crown_vertical_radius = tree.vert_crown_radius

        Li_dz = calculate_Li_dz_per_section(Fa, crown_base_height, crown_radius, crown_center_height, crown_vertical_radius, tree_height, plot.tree_height_array, section_range)

        # # Print all values on one line
        # print(f"tree_id: {tree_id}, "
        #     f"tree_height: {tree_height}, "
        #     f"max tree height: {max_tree_height}, "
        #     f"section range: {section_range}, "
        #     f"num sections: {num_sections}, "
        #     f"length of tree height array: {len(plot.tree_height_array)}, "
            # f"crown_volume: {crown_volume}, "
            # f"crown_base_height: {crown_base_height}, "
            # f"crown_radius (horizontal): {crown_radius}, "
            # f"crown_center_height: {crown_center_height}, "
            # f"crown_vertical_radius: {crown_vertical_radius}, "
            # f"LAI: {LAI}, "
            # f"Li_dz shape: {Li_dz.shape}, "
            # f"Li_dz_indiv[{index}] shape: {plot.Li_dz_indiv[index].shape}")

        # print(Li_dz)
        plot.Li_dz_indiv[index, :len(Li_dz)] = Li_dz

def calculate_Li_dz_per_section(Fa, crown_base_height, crown_radius, crown_center_height, crown_vertical_radius, tree_height, tree_height_array, section_range):
    # Calculate the number of sections
    num_sections = len(tree_height_array)
    
    # Initialize NumPy arrays to store leaf area, veg area, and volume per range height
    Li_dz_per_range = np.zeros(num_sections, dtype=float)

    # Calculate the leaf area, veg area, and volume for the rest of the ranges
    for i in range(int(crown_base_height / section_range), int(tree_height / section_range) + 1):
        Li_dz_per_range_single = Fa * math.pi * crown_radius**2 * abs(1- ((tree_height_array[i] - crown_center_height)/crown_vertical_radius)**2)
        Li_dz_per_range[i] = Li_dz_per_range_single

    return Li_dz_per_range

def process_plots(teak_plot_objects, teak_tree_objects, plot_LAI_df, section_range, plot_area):
    for plot in teak_plot_objects:
        matching_row = plot_LAI_df[plot_LAI_df["namedLocation"].str.contains(plot.plotid, na=False)]
        # plot.LAI = matching_row["LAI_Average_exclude_negatives"].values[0]
        plot.LAI = matching_row["small_footprint_lidar_LAI"].values[0]
        # print(f"plotid: {plot.plotid}, LAI: {plot.LAI}")
        calculate_Li_dz(plot, teak_tree_objects, section_range, plot_area)
        plot.Li_dz_total = np.sum(plot.Li_dz_indiv, axis=0) / plot_area
        plot.acc_LAI = np.cumsum(plot.Li_dz_total[::-1])[::-1] * section_range

def calc_gap_and_wvfm(plot_objects):

    for plot in plot_objects:
        acc_LAI = plot.acc_LAI

        plot.gap = np.exp(-acc_LAI / 2)

        # Set the gap probability at the ground level based on the maximum acc_LAI
        # max_acc_LAI = np.max(acc_LAI)  # Obtain the maximum value of acc_LAI
        # plot.gap[0] = np.exp(- (max_acc_LAI * 1.1) / 2)

        plot.waveform = np.zeros_like(plot.gap)

        for i in range(len(plot.gap)):
            if i == 0:  # First element
                plot.waveform[i] = (plot.gap[i]) / section_range
            elif i == len(plot.gap) - 1:  # Last element
                plot.waveform[i] = (plot.gap[i] - plot.gap[i - 1]) / section_range
            else:  # All other elements
                plot.waveform[i] = (plot.gap[i + 1] - plot.gap[i - 1]) / (section_range * 2)

In [558]:
process_plots(teak_plot_objects_100, teak_tree_objects_100, plot_LAI_df, section_range, 400)
process_plots(teak_plot_objects_400, teak_tree_objects_400, plot_LAI_df, section_range, 800)
calc_gap_and_wvfm(teak_plot_objects_100)
calc_gap_and_wvfm(teak_plot_objects_400)

# print(len(teak_plot_objects_100))
# print(len(teak_plot_objects_400))

plot id: TEAK_001, num_trees: 11, max_tree_height: 23.9, LAI: 0.4095888252920266, total crown vol: 1269.505024945021, Fa: 0.1290546527170351
plot id: TEAK_002, num_trees: 14, max_tree_height: 49.3, LAI: 3.228900908515289, total crown vol: 7398.22911220213, Fa: 0.17457696211055493
plot id: TEAK_003, num_trees: 17, max_tree_height: 25.1, LAI: 0.9690166308972346, total crown vol: 1644.4197600712837, Fa: 0.2357102862483795
plot id: TEAK_005, num_trees: 10, max_tree_height: 24.7, LAI: 1.4556183245217271, total crown vol: 2904.141446614005, Fa: 0.20048862650527727
plot id: TEAK_006, num_trees: 12, max_tree_height: 27.4, LAI: 1.1292667198548043, total crown vol: 1768.8833778212038, Fa: 0.2553626166685476
plot id: TEAK_007, num_trees: 12, max_tree_height: 22.5, LAI: 0.9605326423817256, total crown vol: 1512.2906649708498, Fa: 0.25406032441527776
plot id: TEAK_010, num_trees: 4, max_tree_height: 36.7, LAI: 1.258467709632585, total crown vol: 1208.9250654873183, Fa: 0.4163922961181373
plot id: T

In [559]:
def debug_teak_plots(teak_plot_objects):
    for plot in teak_plot_objects:
        print(f"Plot ID: {plot.plotid}")
        for i in range(len(plot.tree_height_array)):
            height = plot.tree_height_array[i]
            # print(height)
            Li_dz_total = plot.Li_dz_total[i] if i < len(plot.Li_dz_total) else None
            acc_LAI = plot.acc_LAI[i] if i < len(plot.acc_LAI) else None
            gap = plot.gap[i] if i < len(plot.gap) else None
            waveform = plot.waveform[i] if i < len(plot.waveform) else None
            
            # print(f"Height: {height}, Li_dz_total: {Li_dz_total}, acc_LAI: {acc_LAI}, gap: {gap}, waveform: {waveform}")
        print(f"num trees: {len(plot.tree_individualID)}")
        # print("\n")  # Add a new line for better readability between plots

debug_teak_plots(teak_plot_objects_100)
# debug_teak_plots(teak_plot_objects_400)

Plot ID: TEAK_001
num trees: 11
Plot ID: TEAK_002
num trees: 14
Plot ID: TEAK_003
num trees: 17
Plot ID: TEAK_005
num trees: 10
Plot ID: TEAK_006
num trees: 12
Plot ID: TEAK_007
num trees: 12
Plot ID: TEAK_010
num trees: 4
Plot ID: TEAK_011
num trees: 8
Plot ID: TEAK_012
num trees: 41
Plot ID: TEAK_013
num trees: 13
Plot ID: TEAK_014
num trees: 6
Plot ID: TEAK_015
num trees: 45
Plot ID: TEAK_016
num trees: 6
Plot ID: TEAK_017
num trees: 12
Plot ID: TEAK_018
num trees: 6
Plot ID: TEAK_019
num trees: 23
Plot ID: TEAK_020
num trees: 14
Plot ID: TEAK_025
num trees: 4


In [560]:
def plot_teak_data(teak_plot_objects, pgap_list, orig_gort_path, nrows, ncols, suptitle, save_path, plot_type='fp', consistent_ylim=False):
    fig, axs = plt.subplots(nrows, ncols, figsize=(20, 24))
    axs = axs.flatten()  # Flatten the array of axes for easy indexing

    orig_gort_list = find_gort_out_files(orig_gort_path)
    
    # Determine the data to plot based on plot_type
    if plot_type == 'fp':
        data_label = "Foliage Density $m^2$/$m^3$"
        get_data = lambda plot: plot.Li_dz_total
    elif plot_type == 'accLAI':
        data_label = "Accumulated LAI"
        get_data = lambda plot: plot.acc_LAI
    elif plot_type == 'gap':
        data_label = "Gap"
        get_data = lambda plot: plot.gap
    elif plot_type == 'waveform':
        data_label = "Waveform"
        get_data = lambda plot: plot.waveform
    else:
        raise ValueError("Invalid plot_type. Choose from 'fp', 'accLAI', 'gap', or 'waveform'.")

    # Set y-axis limits if consistent_ylim is True
    if consistent_ylim:
        all_heights = np.concatenate([plot.tree_height_array for plot in teak_plot_objects])
        ymin, ymax = all_heights.min(), all_heights.max()

    for i, plot in enumerate(teak_plot_objects):
        matching_gort_file = next((item for item in orig_gort_list if item['name'] == plot.plotid), None)

        if matching_gort_file:
            # print(f'gort file found: {matching_gort_file}')
            
            # Call the read_gort_out_data function with the path of the matching file
            ht, fp, efp, pgap, groups = read_gort_out_data(matching_gort_file['path'])

            # fp should be modified depending on the number of groups            
            if groups == 1:
                fp = fp * 2
            if groups == 2:
                fp = fp
            if groups == 3:
                fp = (fp * 2)/3
                
            # Determine the data to plot based on plot_type
            if plot_type == 'fp':
                axs[i].plot(fp, ht, label='Original GORT Data fp/efp', color='r', linestyle='--')
                axs[i].set_xlim(0, 0.22)
            elif plot_type == 'accLAI':
                accLAI = np.cumsum(fp[::-1])[::-1] * section_range
                print(accLAI)
                axs[i].plot(accLAI, ht, label='Original GORT Data accLAI', color='r', linestyle=':')
                # axs[i].set_xlim(0, 4)
            elif plot_type == 'gap':
                axs[i].plot(pgap, ht, label='Original GORT Data pgap', color='r', linestyle='-.')
                # axs[i].set_xlim(0, 1.1)
            
        height = plot.tree_height_array
        axs[i].plot(get_data(plot), height, label='New GORT Data', color='b')

        # Find matching pgap data based on plotid
        pgap_data = next((pgap for pgap, plotid in pgap_list if plotid == plot.plotid), None)

        if pgap_data:
            # Extract foliageDensity, gap, and accLAI before the if/elif block
            mask = (pgap_data.height > 0.1) & (~np.isnan(pgap_data.foliageDensity)) # exclude all data below height 0.1, as they are too large.  Also filter out NaN values
            pgap_ht = pgap_data.height[mask]
            pgap_fp = pgap_data.foliageDensity[mask]
            pgap_gap = pgap_data.gap[mask]
            
            # Calculate the height differences (dz)
            height_diffs = np.abs(np.diff(pgap_ht, prepend=pgap_ht[0]))

            pgap_acc_LAI = np.cumsum(pgap_fp * height_diffs) # Cumulative sum multiplied by dz
            
            # Plot data based on plot_type
            if plot_type == 'fp':
                print(np.nanmax(pgap_fp))
                axs[i].plot(pgap_fp, pgap_ht, label='Small Footprint fp', color='green')
            elif plot_type == 'accLAI':
                axs[i].plot(pgap_acc_LAI, pgap_ht, label='Small Footprint accLAI', color='green')
            elif plot_type == 'gap':
                axs[i].plot(pgap_gap, pgap_ht, label='Small footprint Gap', color='green')
        
        if consistent_ylim:
            axs[i].set_ylim(ymin, ymax)
            
        axs[i].set_title(plot.plotid)
        axs[i].set_xlabel(data_label)
        axs[i].set_ylabel("Height (m)")
        axs[i].grid()
        axs[i].legend()
        axs[i].tick_params(axis='x', rotation=45)

    for j in range(i + 1, nrows * ncols):
        axs[j].axis('off')

    plt.suptitle(suptitle)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    
    # Create the full path
    full_save_path = os.path.join(save_path, date_subfolder, plot_type)
    # Create directories if they don't exist
    os.makedirs(full_save_path, exist_ok=True)
    # Save the figure
    fig.savefig(os.path.join(full_save_path, f"{suptitle}.png"), bbox_inches='tight')
    print(f"fig saved to: {full_save_path}/{suptitle}.png")

    plt.show()
    
def find_gort_out_files(directory):
    txt_files_list = []
    
    # Traverse through the directory using os.walk()
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file ends with .txt and matches the pattern
            if file.endswith(".out"):
                # Strip the file extension and the extra parts (.in.gwvfm.txt)
                stripped_name = file.split(".out")[0]
                
                # Create a dictionary with stripped name and file path
                file_dict = {
                    'name': stripped_name,
                    'path': os.path.join(root, file)
                }
                
                # Append the dictionary to the list
                txt_files_list.append(file_dict)

    return txt_files_list

def read_gort_out_data(gort_out_location):
    height = []
    fp = []
    efp = []
    pgap = []
    
    gort_in_location = gort_out_location.replace(".out", ".in")
    
    with open(gort_in_location) as gort_in:
        first_line = gort_in.readline().strip()
    
    groups = int(first_line)
    
    with open(gort_out_location) as idl:
        for line in idl:
            if 'gamma:' in line:
                print('Found "gamma:" line')
                break  # Stop reading once we find the "gamma:" line
            
        for line in idl:
            parts = line.split()
            
            # Check if the line contains exactly 6 columns (assuming this is the data you need)
            if len(parts) == 6:
                # Start reading from this line
                height.append(float(parts[0]))
                fp.append(float(parts[1]))
                efp.append(float(parts[2]))
                pgap.append(float(parts[3]))
    
    return np.array(height), np.array(fp), np.array(efp), np.array(pgap), groups

In [ ]:
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['axes.labelsize'] = 24  # Size for x and y axis labels
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['figure.titlesize'] = 30

# plot_teak_data(teak_plot_objects_100, pgap_list, orig_gort_path, 4, 5, "100 m2 plots FP", output_path, plot_type='fp', consistent_ylim=True)
# plot_teak_data(teak_plot_objects_400, pgap_list, orig_gort_path, 2, 3, "400 m2 plots FP", output_path, plot_type='fp', consistent_ylim=True)

combined_plot_objects = teak_plot_objects_100 + teak_plot_objects_400

plot_teak_data(combined_plot_objects, pgap_list, orig_gort_path, 5, 5, "Combined 100 and 400 m2 plots FP", output_path, plot_type='fp', consistent_ylim=True)

In [ ]:
# plot_teak_data(teak_plot_objects_100, pgap_list, orig_gort_path, 4, 5, "100 m2 Acc LAI", output_path, plot_type='accLAI', consistent_ylim=True)
# plot_teak_data(teak_plot_objects_400, pgap_list, orig_gort_path, 2, 3, "400 m2 Acc LAI", output_path, plot_type='accLAI', consistent_ylim=True)


plot_teak_data(combined_plot_objects, pgap_list, orig_gort_path, 5, 5, "Combined 100 and 400 m2 plots Acc LAI", output_path, plot_type='accLAI', consistent_ylim=True)

In [ ]:
# plot_teak_data(teak_plot_objects_100, pgap_list, orig_gort_path, 4, 5, "100 m2 plots Gap Prob", output_path, plot_type='gap', consistent_ylim=True)
# plot_teak_data(teak_plot_objects_400, pgap_list, orig_gort_path, 2, 3, "400 m2 plots Gap Prob", output_path, plot_type='gap', consistent_ylim=True)

plot_teak_data(combined_plot_objects, pgap_list, orig_gort_path, 5, 5, "Combined 100 and 400 m2 plots Gap", output_path, plot_type='gap', consistent_ylim=True)

In [ ]:
# plot_teak_data(teak_plot_objects_100, pgap_list, orig_gort_path, 4, 5, "100 m2 plots Waveform", output_path, plot_type='waveform', consistent_ylim=True)
# plot_teak_data(teak_plot_objects_400, pgap_list, orig_gort_path, 2, 3, "400 m2 plots Waveform", output_path, plot_type='waveform', consistent_ylim=True)

plot_teak_data(combined_plot_objects, pgap_list, orig_gort_path, 5, 5, "Combined 100 and 400 m2 plots waveform", output_path, plot_type='waveform', consistent_ylim=True)

In [565]:
importlib.reload(TEAK_class_declarations)

def gsmooth(values, width, peak_scale=1.0):
    # Apply scaling factor to the highest peaks
    max_value = np.max(values)
    scaled_values = np.copy(values)
    scaled_values[values >= max_value * 0.9] *= peak_scale
    
    # Calculate convolution parameters and perform convolution
    N = len(scaled_values)
    N2 = int(np.ceil(width * 8))
    if N2 > N / 2:
        N2 = N // 2
    N2 = int(N2 / 2) * 2 + 1
    g1 = np.exp(-np.arange(-N2//2, N2//2)**2 / (2.0 * width**2))
    g1 = g1 / np.sum(g1)
    
    smoothed_values = convolve(scaled_values, g1, mode='constant', cval=0.0)
    
    return smoothed_values

def convolute_and_apply_smoothing(plot_objects, peak_scale=0.95):
    
    width = int(0.6 / section_range) + 1

    for plot in plot_objects:
        n_ext = int(10.0 / section_range)
        height_array = plot.tree_height_array
        waveform = plot.waveform

        new_height_array = np.zeros(len(height_array) + n_ext, dtype=np.float32)
        gsmooth_array = np.zeros(len(waveform) + n_ext, dtype=np.float32)
        gaussian_array = np.zeros(len(waveform) + n_ext, dtype=np.float32)

        for height_level in range(len(waveform)):
            new_height_array[height_level + n_ext] = height_array[height_level]
            gsmooth_array[height_level + n_ext] = waveform[height_level]
            gaussian_array[height_level + n_ext] = waveform[height_level]

        new_height_array[:n_ext] = np.arange(n_ext) * section_range - 10

        plot.extended_height_array = new_height_array
        # Apply custom gsmooth function
        plot.gsmooth_wvfm = gsmooth(gsmooth_array, width, peak_scale)
        # Apply scipy's gaussian_filter function
        plot.gaussian_wvfm = gaussian_filter(gaussian_array, sigma=width, mode='nearest')

In [566]:
convolute_and_apply_smoothing(teak_plot_objects_100)
convolute_and_apply_smoothing(teak_plot_objects_400)

In [567]:
def plot_teak_waveforms(teak_plot_objects, pgap_list, orig_gort_path, nrows, ncols, suptitle, save_path, consistent_ylim=False, consistent_xlim=False):
    fig, axs = plt.subplots(nrows, ncols, figsize=(20, 20))
    axs = axs.flatten()  # Flatten the array of axes for easy indexing

    orig_gort_list = find_txt_files(orig_gort_path)
    
    # Set y-axis limits if consistent_ylim is True
    if consistent_ylim:
        all_heights = np.concatenate([plot.extended_height_array for plot in teak_plot_objects])
        ymin, ymax = all_heights.min(), all_heights.max()

    for i, plot in enumerate(teak_plot_objects):
        # Check if plot.plotid exists in the orig_gort_list
        matching_gort_file = next((item for item in orig_gort_list if item['name'] == plot.plotid), None)

        if matching_gort_file:
            # Call the read_idl_waveform_data function with the path of the matching file
            idl_waveform_ght, idl_waveform = read_idl_waveform_data(matching_gort_file['path'])
            
            # print(f"plotid: {plot.plotid}, file found: {matching_gort_file['path']}")
            # Plot the GORT waveform data
            axs[i].plot(idl_waveform, idl_waveform_ght, label='Original GORT Data', color='y', linestyle='-.')
            # for x, val in enumerate(idl_waveform):
                # print(f"height: {idl_waveform_ght[x]}, wvm_val: {val}")
        
        # Plot the waveform (original, gsmooth, and gaussian) vs height
        axs[i].plot(plot.waveform, plot.tree_height_array, label='Raw Waveform', color='b')
        axs[i].plot(plot.gsmooth_wvfm, plot.extended_height_array, label='Custom Gsmooth Function', color='g', linestyle='--')
        axs[i].plot(plot.gaussian_wvfm, plot.extended_height_array, label='SciPy gaussian_filter', color='r', linestyle=':')

        # Find matching pgap data based on plotid
        pgap_data = next((pgap for pgap, plotid in pgap_list if plotid == plot.plotid), None)

        if pgap_data:
            # Extract foliageDensity, gap, and accLAI before the if/elif block
            mask = (pgap_data.height > 0.1) & (~np.isnan(pgap_data.foliageDensity)) # exclude all data below height 0.1, as they are too large.  Also filter out NaN values
            pgap_ht = pgap_data.height[mask]
            pgap_gap = pgap_data.gap[mask]

            pgap_wvfm = calc_smallfootprint_waveform(pgap_gap, pgap_ht)
            
            pgap_smoothed_wvfm, pgap_ext_ht = smooth_pgap_waveform(pgap_wvfm, pgap_ht)
            
            # axs[i].plot(pgap_smoothed_wvfm, pgap_ext_ht, label='Small Footprint pseudo wvfm', color='brown', linestyle='-')
            axs[i].plot(pgap_smoothed_wvfm, pgap_ext_ht, label='Small Footprint pseudo wvfm', color='brown', linestyle='-')

        axs[i].set_title(plot.plotid)
        axs[i].set_xlabel("Waveform Values")
        axs[i].set_ylabel("Height (m)")
        axs[i].grid()
        axs[i].legend()
        axs[i].tick_params(axis='x', rotation=45)

        if consistent_ylim:
            axs[i].set_ylim(ymin, ymax)
        
        if consistent_xlim:
            axs[i].set_xlim(0, 0.3)

    # Turn off extra subplots if necessary
    for j in range(i + 1, nrows * ncols):
        axs[j].axis('off')

    plt.suptitle(suptitle)
    plt.tight_layout(rect=[0, 0, 1, 0.97])

    # Create the full path for saving the plot
    full_save_path = os.path.join(save_path, date_subfolder, "combined_waveforms")
    os.makedirs(full_save_path, exist_ok=True)
    fig.savefig(os.path.join(full_save_path, f"{suptitle}.png"), bbox_inches='tight')
    print(f"fig saved to: {full_save_path}/{suptitle}.png")

    plt.show()

def calc_smallfootprint_waveform(gap_array, height_array):
    # Calculate height differences (dz)
    height_diffs = np.diff(height_array, prepend=height_array[0])
    if len(height_diffs) > 0:
        height_diffs[0] = height_diffs[1]
    
    # Initialize the waveform array with zeros
    waveform_array = np.zeros_like(gap_array)

    # Calculate the waveform based on the gap array
    for i in range(len(gap_array)):
        if i == 0:  # First element
            waveform_array[i] = gap_array[i] / height_diffs[i]
        elif i == len(gap_array) - 1:  # Last element
            waveform_array[i] = (gap_array[i] - gap_array[i - 1]) / height_diffs[i]
        else:  # All other elements
            waveform_array[i] = (gap_array[i + 1] - gap_array[i - 1]) / (height_diffs[i] + height_diffs[i - 1]) 

    return waveform_array

def find_txt_files(directory):
    txt_files_list = []
    
    # Traverse through the directory using os.walk()
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file ends with .txt and matches the pattern
            if file.endswith(".txt"):
                # Strip the file extension and the extra parts (.in.gwvfm.txt)
                stripped_name = file.split(".in.gwvfm.txt")[0]
                
                # Create a dictionary with stripped name and file path
                file_dict = {
                    'name': stripped_name,
                    'path': os.path.join(root, file)
                }
                
                # Append the dictionary to the list
                txt_files_list.append(file_dict)

    return txt_files_list

def read_idl_waveform_data(idl_wvfm_location):
    gwvform_ght = []
    gwvform = []
    
    with open(idl_wvfm_location) as idl:
        for i, line in enumerate(idl):
            if i < 4:
                continue  # Skip the first four lines
            parts = line.split()
            
            gwvform_ght.append(float(parts[0]))
            gwvform.append(float(parts[1]))
    
    return np.array(gwvform_ght), np.array(gwvform)

def smooth_pgap_waveform(waveform, height_array, peak_scale=0.95):
    # Calculate section_range (dz) from the height array
    section_range = np.abs(np.diff(height_array, prepend=height_array[0]))
    if len(section_range) > 0:
        section_range[0] = section_range[1]
    
    # Calculate the width based on the section range
    width = int(0.6 / section_range[0]) + 1  # Assuming uniform section range

    # Extend the waveform array
    n_ext = int(10.0 / section_range[0])
    extended_waveform = np.zeros(len(waveform) + n_ext, dtype=np.float32)

    # Fill the extended waveform
    extended_waveform[n_ext:n_ext + len(waveform)] = waveform
    extended_waveform[:n_ext] = np.arange(n_ext) * section_range[0] - 10  # Example filling for extension

    # Extend the height array
    extended_height_array = np.zeros(len(height_array) + n_ext, dtype=np.float32)
    extended_height_array[n_ext:n_ext + len(height_array)] = height_array
    extended_height_array[:n_ext] = np.arange(n_ext) * section_range[0] - 10  # Example filling for extension

    # Apply Gaussian filter to the extended waveform
    smoothed_waveform = gaussian_filter(extended_waveform, sigma=width, mode='nearest')
    
    return smoothed_waveform, extended_height_array

In [ ]:
# plot_teak_waveforms(teak_plot_objects_100, pgap_list, orig_gort_path, 4, 5, "100 m2 Combined Waveforms", output_path, consistent_ylim=True, consistent_xlim=True)
# plot_teak_waveforms(teak_plot_objects_400, pgap_list, orig_gort_path, 2, 3, "400 m2 Combined Waveforms", output_path, consistent_ylim=True, consistent_xlim=True)

plot_teak_waveforms(combined_plot_objects, pgap_list, orig_gort_path, 5, 5, "100 and 400 m2 Combined Waveforms", output_path, consistent_ylim=True, consistent_xlim=True)

In [592]:

def plot_teak_waveforms(teak_plot_objects, pgap_list, orig_gort_path, nrows, ncols, suptitle, save_path, consistent_ylim=False, consistent_xlim=False):
    fig, axs = plt.subplots(nrows, ncols, figsize=(20, 25))
    axs = axs.flatten()  # Flatten the array of axes for easy indexing

    orig_gort_list = find_gort_out_files(orig_gort_path)

    # Initialize empty lists to collect handles and labels for the global legend
    handles, labels = [], []
    
    # Create empty list to collect all heights (to calculate ymax later)
    all_heights = []
    all_pgap_heights = []

    for i, plot in enumerate(teak_plot_objects):
        # Check if plot.plotid exists in the orig_gort_list
        matching_gort_file = next((item for item in orig_gort_list if item['name'] == plot.plotid), None)

        if matching_gort_file:
            # Call the read_idl_waveform_data function with the path of the matching file
            height_array, smoothed_waveform = process_out_file(matching_gort_file['path'])
            
            # print(f"plotid: {plot.plotid}, file found: {matching_gort_file['path']}")
            # Plot the GORT waveform data
            line, = axs[i].plot(smoothed_waveform, height_array, label='Original GORT Data', color='r', linestyle='-.')
            if 'Original GORT Data' not in labels:
                handles.append(line)
                labels.append('Original GORT Data')
            # for x, val in enumerate(idl_waveform):
                # print(f"height: {idl_waveform_ght[x]}, wvm_val: {val}")
        
        # Plot the waveform (original, gsmooth, and gaussian) vs height
        line, = axs[i].plot(plot.gsmooth_wvfm, plot.extended_height_array, label='New GORT Data', color='b', linestyle='--')
        if 'New GORT Data' not in labels:
            handles.append(line)
            labels.append('New GORT Data')
        # Append the heights to the all_heights list
        all_heights.extend(plot.extended_height_array)

        # Find matching pgap data based on plotid
        pgap_data = next((pgap for pgap, plotid in pgap_list if plotid == plot.plotid), None)

        if pgap_data:
            # Extract foliageDensity, gap, and accLAI before the if/elif block
            mask = (pgap_data.height > 0.1) & (~np.isnan(pgap_data.foliageDensity)) # exclude all data below height 0.1, as they are too large.  Also filter out NaN values
            pgap_ht = pgap_data.height[mask]
            pgap_gap = pgap_data.gap[mask]

            pgap_wvfm = calc_smallfootprint_waveform(pgap_gap, pgap_ht)
            
            pgap_smoothed_wvfm, pgap_ext_ht = smooth_pgap_waveform(pgap_wvfm, pgap_ht)
            
            # Append the pgap_ext_ht to the all_pgap_heights list
            all_pgap_heights.extend(pgap_ext_ht)
            
            # axs[i].plot(pgap_smoothed_wvfm, pgap_ext_ht, label='Small Footprint pseudo wvfm', color='brown', linestyle='-')
            line, = axs[i].plot(pgap_smoothed_wvfm, pgap_ext_ht, label='Small Footprint pseudo wvfm', color='green', linestyle='-')
            if 'Small Footprint pseudo wvfm' not in labels:
                handles.append(line)
                labels.append('Small Footprint pseudo wvfm')

        axs[i].set_title(plot.plotid)
        axs[i].set_xlabel("Waveform Values")
        axs[i].set_ylabel("Height (m)")
        axs[i].grid()
        axs[i].tick_params(axis='x', rotation=45)
        
        if consistent_xlim:
            axs[i].set_xlim(0, 0.05)

    # After all subplots, calculate the global ymin and ymax
    if consistent_ylim:
        # Combine all heights (from both the plot and pgap data) for the global ymin, ymax calculation
        combined_heights = np.concatenate([all_heights, all_pgap_heights])
        ymin, ymax = combined_heights.min(), combined_heights.max()

        # Apply consistent y-limits across all subplots
        for ax in axs:
            ax.set_ylim(ymin, ymax)

    # Turn off extra subplots if necessary
    for j in range(i + 1, nrows * ncols):
        axs[j].axis('off')

    plt.suptitle(suptitle)
    plt.tight_layout(rect=[0, 0, 1, 0.98])

    # Create the full path for saving the plot
    full_save_path = os.path.join(save_path, date_subfolder, "combined_waveforms")
    os.makedirs(full_save_path, exist_ok=True)
    fig.savefig(os.path.join(full_save_path, f"{suptitle}.png"), bbox_inches='tight')
    print(f"fig saved to: {full_save_path}/{suptitle}.png")

    # Add a legend inside the plot area (bottom right)
    plt.legend(handles=handles, labels=labels, loc='lower right', fontsize=14)

    plt.show()

def process_out_file(out_file):
    
    with open(out_file) as gort_out:
        
        gn_level, dz = map(float, gort_out.readline().split())
        n_ext = int(10.0 / dz)
        n_tot = int(gn_level + n_ext)

        height_array = np.zeros(n_tot, dtype=np.float32)
        waveform_array = np.zeros(n_tot, dtype=np.float32)

        height_array[:n_ext] = np.arange(n_ext) * dz - 10  # extend height to below ground 10m deep
        
        for line in gort_out:
            if 'gamma:' in line:
                # print('Found "gamma:" line')
                break  # Stop reading once we find the "gamma:" line
        
        print(out_file, gn_level, dz, n_tot)
            
        for line in gort_out:
            parts = line.split()
            parts = [float(part) for part in parts]
            
            # Check if the line contains exactly 6 columns (assuming this is the data you need)
            if len(parts) == 6:
                
                # Start reading from this line
                for ilevel in range(int(gn_level)):
                    height_array[ilevel + n_ext] = parts[0]
                    waveform_array[ilevel + n_ext] = parts[5]
                    print(f"line: {line}, parts[0]: {parts[0]}, parts[5]: {parts[5]}")

        # checks only 1 file for debug
        if 'TEAK_001' in out_file:
            print(f"height_array: {height_array}")
            print(f"waveform_array: {waveform_array}")

        width = int(0.6 / dz) + 1
        smoothed_waveform = gaussian_filter(waveform_array, sigma=width, mode='nearest')
        print(len(height_array), len(smoothed_waveform))
        # print(height_array, smoothed_waveform)
    
    return height_array, smoothed_waveform

In [ ]:
plot_teak_waveforms(combined_plot_objects, pgap_list, orig_gort_path, 5, 5, "100 and 400 m2 Combined Waveforms", output_path, consistent_ylim=True, consistent_xlim=True)